In [2]:
import pandas as pd

In [10]:
df1 = pd.read_excel('CarRentalData.xlsx')
df2 = pd.read_excel('Flights.xlsx')
df3 = pd.read_excel('Hotel_details.xlsx')
df4 = pd.read_csv('hotel_price_min_max - Formula.csv')
df5 = pd.read_csv('Hotel_Room_attributes.csv')
df6 = pd.read_csv('hotels_RoomPrice.csv')

**Flight**

In [11]:
df2.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,Birmingham,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Boise,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Nashville,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Oakland,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,Corpus Christi,16:50,21:35,4h 45m,1 stop,No info,13302


In [12]:
df2['Arrival_Time'] = df2['Arrival_Time'].str[:5]

In [15]:
import datetime
import pandas as pd
import psycopg2
from psycopg2 import extras
import numpy as np
from psycopg2 import sql

# Establish a connection to the database
conn = psycopg2.connect(
    dbname="SQL_Final",
    user="postgres",
    password="5k868af",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

for index, row in df2.iterrows():
    # Inserting Source into AirportSource
    cur.execute(
        "INSERT INTO AirportSource (Source) VALUES (%s) ON CONFLICT (Source) DO NOTHING RETURNING SourceID;", 
        (row['Source'],))
    fetched_row = cur.fetchone()
    source_id = fetched_row[0] if fetched_row else None

    # Inserting Destination into AirportDestination
    cur.execute(
        "INSERT INTO AirportDestination (Destination) VALUES (%s) ON CONFLICT (Destination) DO NOTHING RETURNING DestinationID;", 
        (row['Destination'],))
    fetched_row = cur.fetchone()
    destination_id = fetched_row[0] if fetched_row else None

    # Insert into Airline
    cur.execute(
    "INSERT INTO Airline (AirlineName) VALUES (%s) ON CONFLICT (AirlineName) DO NOTHING RETURNING AirlineID;", 
    (row['Airline'],))
    fetched_row = cur.fetchone()
    airline_id = fetched_row[0] if fetched_row else None

    # Insert into Route
    cur.execute(
        "INSERT INTO Route (SourceID, DestinationID, Dep_Time, Arrival_Time, Duration) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (SourceID, DestinationID, Dep_Time, Arrival_Time, Duration) DO NOTHING RETURNING RouteID;", 
        (source_id, destination_id, row['Dep_Time'], row['Arrival_Time'], row['Duration']))
    fetched_row = cur.fetchone()
    route_id = fetched_row[0] if fetched_row else None

    # Converting the Date_of_Journey from string to date
    date_of_journey = datetime.datetime.strptime(row['Date_of_Journey'], "%d/%m/%Y").date()

    # Insert into FlightBooking
    cur.execute(
        "INSERT INTO FlightBooking (AirlineID, RouteID, Date_of_Journey, Price) VALUES (%s, %s, %s, %s);", 
        (airline_id, route_id, date_of_journey, row['Price']))
    
    # Commit each row insert
    conn.commit()

# Close the connection
cur.close()
conn.close()

**CarRental**

In [33]:
df1.head()

,fuelType,rating,renterTripsTaken,reviewCount,location.city,location.country,location.latitude,location.longitude,location.state,rate.daily,vehicle.make,vehicle.model,vehicle.type,vehicle.year,airportcity
0,ELECTRIC,5.00,13,12,Seattle,US,47.449107,-122.308841,WA,135,Tesla,Model X,suv,2019,Albuquerque
1,ELECTRIC,5.00,2,1,Tijeras,US,35.111060,-106.276551,NM,190,Tesla,Model X,suv,2018,Albuquerque
2,HYBRID,4.92,28,24,Albuquerque,US,35.127163,-106.566681,NM,35,Toyota,Prius,car,2012,Albuquerque
3,GASOLINE,5.00,21,20,Albuquerque,US,35.149726,-106.711425,NM,75,Ford,Mustang,car,2018,Albuquerque
4,GASOLINE,5.00,3,1,Albuquerque,US,35.208659,-106.601008,NM,47,Chrysler,Sebring,car,2010,Albuquerque


In [38]:

# Establish a connection to the database
conn = psycopg2.connect(
    dbname="SQL_Final",
    user="postgres",
    password="5k868af",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

for index, row in df1.iterrows():
    # Insert into Fuel table
    cur.execute(
    "INSERT INTO Fuel (fuel_type) VALUES (%s) ON CONFLICT (fuel_type) DO NOTHING RETURNING fuel_id;", (row['fuelType'],))
    fetched_row = cur.fetchone()
    fuel_id = fetched_row[0] if fetched_row else None

    # Insert into Location_Car table
    cur.execute(
    "INSERT INTO Location_Car (city, country, latitude, longitude, state) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (city, country, state) DO NOTHING RETURNING location_id;", 
    (row['location.city'], row['location.country'], row['location.latitude'], row['location.longitude'], row['location.state']))

    fetched_row = cur.fetchone()
    location_id = fetched_row[0] if fetched_row else None


    # Insert into Vehicle table
    cur.execute(
        "INSERT INTO Vehicle (make, model, vehicle_type, vehicle_year) VALUES (%s, %s, %s, %s) ON CONFLICT (make, model, vehicle_type, vehicle_year) DO NOTHING RETURNING vehicle_id;", 
        (row['vehicle.make'], row['vehicle.model'], row['vehicle.type'], int(row['vehicle.year']) if pd.notna(row['vehicle.year']) else None))
    fetched_row = cur.fetchone()
    vehicle_id = fetched_row[0] if fetched_row else None

    # Fetch the DestinationID for the given airport city
    cur.execute("SELECT DestinationID FROM AirportDestination WHERE Destination = %s;", (row['airportcity'],)) # Modified this line
    result = cur.fetchone()
    airport_city_id = result[0] if result else None

    # Check if a valid airport_city_id was retrieved
    if airport_city_id is None:
        print(f"Error: Could not find DestinationID for city: {row['airportcity']}")
        continue

    # Insert into CarRental table
    cur.execute(
        "INSERT INTO CarRental (renterTripsTaken, reviewCount, rate_daily, rating, vehicle_id, location_id, fuel_id, Airport_cityID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);", 
        (float(row['renterTripsTaken']) if pd.notna(row['renterTripsTaken']) else None, 
         int(row['reviewCount']) if pd.notna(row['reviewCount']) else None, 
         float(row['rate.daily']) if pd.notna(row['rate.daily']) else None, 
         float(row['rating']) if pd.notna(row['rating']) else None, 
         vehicle_id, location_id, fuel_id, airport_city_id))

    # Commit each row insert
    conn.commit()

# Close the connection
cur.close()
conn.close()

**Hotel**

In [39]:
df3.head()

,id,hotelid,hotelname,city,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Valparaiso,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Pittsburgh,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,Newark,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Tallahassee,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Tampa,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [40]:
df4.head()

,hotelcode,min,max,Diff_Min,Diff_Max,Score
0,31,75.10,189.88,0.000,0.000,0.00
1,97,259.76,400.00,1.103,0.712,0.91
2,834,220.67,656.86,0.786,1.124,0.95
3,3144,254.54,347.30,0.886,0.395,0.64
4,11203,230.91,244.58,0.748,0.149,0.45


In [41]:
df5.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


In [42]:
df6.head()

,id,refid,hotelcode,websitecode,dtcollected,ratedate,los,guests,roomtype,onsiterate,...,promoname,status_code,taxstatus,taxtype,taxamount,proxyused,israteperstay,mealinclusiontype,hotelblock,input_dtcollected
0,50677497,8646773,634876,5,2019-10-12 15:46:54,2019-11-02,1,1,Double Room,82.36,...,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.165.126:80,Y,Free Breakfast,NaN,2019-10-13
1,50672149,6234691,8328096,5,2019-10-12 15:47:45,2019-11-30,1,1,Vacation Home,636.09,...,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.164.126:80,Y,NaN,NaN,2019-10-13
2,50643430,7015677,8323442,5,2019-10-12 15:47:00,2019-12-20,1,1,Vacation Home,591.74,...,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.166.122:80,Y,NaN,NaN,2019-10-13
3,50650317,7327094,7975,5,2019-10-12 15:47:02,2019-12-28,1,1,Standard Triple Room,881.48,...,NaN,200,2,Tax 20%,0.0,media:M3diAproxy@209.242.219.141:80,Y,NaN,NaN,2019-10-13
4,50650318,7327094,7975,5,2019-10-12 15:47:02,2019-12-28,1,1,Standard Triple Room,897.53,...,NaN,200,2,VAT $ 179.51,0.0,media:M3diAproxy@209.242.219.141:80,Y,Free breakfast for {3},NaN,2019-10-13


In [45]:
# Establish a connection to the database
conn = psycopg2.connect(
    dbname="SQL_Final",
    user="postgres",
    password="5k868af",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Loading data from df3 to the hotels table
for index, row in df3.iterrows():
    cur.execute(
        "SELECT hotelcode FROM hotels WHERE hotelcode = %s", 
        (row['hotelid'],)
    )
    result = cur.fetchone()
    
    # Fetch the DestinationID for the given city from AirportDestination
    cur.execute("SELECT DestinationID FROM AirportDestination WHERE Destination = %s;", (row['city'],))
    city_result = cur.fetchone()
    city_id = city_result[0] if city_result else None

    # If a valid city_id was not retrieved, skip this row
    if city_id is None:
        print(f"Error: Could not find DestinationID for city: {row['city']}")
        continue

    # If the hotelcode is not in the database, insert it
    if result is None:
        cur.execute(
            "INSERT INTO hotels (hotelcode, hotelname, cityID, propertytype, starrating, url) VALUES (%s, %s, %s, %s, %s, %s)",
            (row['hotelid'], row['hotelname'], city_id, row['propertytype'], row['starrating'], row['url'])
        )
    else:
        # If it's already in the database, update the record
        cur.execute(
            "UPDATE hotels SET hotelname = %s, cityID = %s, propertytype = %s, starrating = %s, url = %s WHERE hotelcode = %s",
            (row['hotelname'], city_id, row['propertytype'], row['starrating'], row['url'], row['hotelid'])
        )

# Loading data from df4 to the hotelprices table
for index, row in df4.iterrows():
    cur.execute(
        "SELECT hotelcode FROM hotelprices WHERE hotelcode = %s", 
        (row['hotelcode'],)
    )
    result = cur.fetchone()
    
    # If the hotelcode is not in the database, insert it
    if result is None:
        cur.execute(
            "INSERT INTO hotelprices (hotelcode, min, max, score) VALUES (%s, %s, %s, %s)",
            (row['hotelcode'], row['min'], row['max'], row['Score'])
        )
    else:
        # If it's already in the database, you can choose to update the record
        cur.execute(
            "UPDATE hotelprices SET min = %s, max = %s, score = %s WHERE hotelcode = %s",
            (row['min'], row['max'], row['Score'], row['hotelcode'])
        )
        
        
# Loading data from df5 to the hotel_room_amenities and rooms tables
for index, row in df5.iterrows():
    # Insert into hotel_room_amenities first to get the id for rooms table
    cur.execute(
        "INSERT INTO hotel_room_amenities (hotelcode, roomamenities) VALUES (%s, %s) RETURNING id",
        (row['hotelcode'], row['roomamenities'])
    )
    amenities_id = cur.fetchone()[0]

    # Now insert into rooms
    cur.execute(
        "INSERT INTO rooms (hotelcode, hotel_room_amenities_id, roomtype, ratedescription) VALUES (%s, %s, %s, %s)",
        (row['hotelcode'], amenities_id, row['roomtype'], row['ratedescription'])
    )

# Loading data from df6 to the reviews table
for index, row in df6.iterrows():
    cur.execute(
        "INSERT INTO reviews (id, hotelcode, dtcollected, ratedate, guests, onsiterate, ratedescription, ratetype) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
        (row['id'], row['hotelcode'], row['dtcollected'], row['ratedate'], row['guests'], row['onsiterate'], row['ratedescription'], row['ratetype'])
    )

# Committing the transaction
conn.commit()

cur.close()
conn.close()